## Extracción y transformación de datos con Spark 2.4
### Origen de datos
Los datos representan los accidentes de tráfico por día y distrito
que se han producido en la ciudad de Madrid entre 2010 y 2018

El fichero origen es un fichero csv por año tiene los datos detallados
por cada uno de los intervinientes en el accidente: Conductor, pasajero,
testito, peatón, ...

La primera parte del fichero está referida al accidente y se repite
por cada uno de los intervinientes.

### La parte del registro referida al accidente tiene los campos:
 - 'FECHA'
 - 'RANGO HORARIO'
 - 'DIA SEMANA'
 - 'DISTRITO'
 - 'LUGAR ACCIDENTE'
 - 'Nº: string'
 - 'Nº PARTE'     **Id.accidente, es el mismo para varios regs.**
 - 'CPFA Granizo'
 - 'CPFA Hielo'
 - 'CPFA Lluvia'
 - 'CPFA Niebla'
 - 'CPFA Seco'
 - 'CPFA Nieve'
 - 'CPSV Mojada'
 - 'CPSV Aceite'
 - 'CPSV Barro'
 - 'CPSV Grava Suelta'
 - 'CPSV Hielo'
 - 'CPSV Seca Y Limpia'
 - '* Nº VICTIMAS' **Total de víctimas por Nº de parte**
 - 'TIPO ACCIDENTE'
 - 'Tipo Vehiculo'
 
### La parte del registro correspondiente a cada persona:
 - 'TIPO PERSONA'
 - 'SEXO'
 - 'LESIVIDAD'
 - 'Tramo Edad'


In [1]:
import pyspark
from pyspark.sql import SparkSession
app_name = "bubbly"
master = "local[*]"
spark = (SparkSession.builder
    .master(master)
    .config("spark.driver.cores", 1)
    .appName(app_name)
    .getOrCreate() )
sc = spark.sparkContext
print ('SparkContext created')

SparkContext created


In [2]:
# Cargamos el fichero desde hdfs
#accidenteData = spark.read.csv ('hdfs://localhost:9000/user/ubuntu/accidentes/datos',header=True)
#accidenteData = spark.read.csv ('file:///home/ubuntu/Downloads/2018_Accidentalidad.csv',header=True)
accidenteData = spark.read.csv ('file:///home/jose/Descargas/2018.csv',header=True)

In [3]:
accidenteData.printSchema()

root
 |-- FECHA: string (nullable = true)
 |-- RANGO HORARIO: string (nullable = true)
 |-- DIA SEMANA: string (nullable = true)
 |-- DISTRITO: string (nullable = true)
 |-- LUGAR ACCIDENTE: string (nullable = true)
 |-- Nº: string (nullable = true)
 |-- Nº PARTE: string (nullable = true)
 |-- CPFA Granizo: string (nullable = true)
 |-- CPFA Hielo: string (nullable = true)
 |-- CPFA Lluvia: string (nullable = true)
 |-- CPFA Niebla: string (nullable = true)
 |-- CPFA Seco: string (nullable = true)
 |-- CPFA Nieve: string (nullable = true)
 |-- CPSV Mojada: string (nullable = true)
 |-- CPSV Aceite: string (nullable = true)
 |-- CPSV Barro: string (nullable = true)
 |-- CPSV Grava Suelta: string (nullable = true)
 |-- CPSV Hielo: string (nullable = true)
 |-- CPSV Seca Y Limpia: string (nullable = true)
 |-- * Nº VICTIMAS: string (nullable = true)
 |-- TIPO ACCIDENTE: string (nullable = true)
 |-- Tipo Vehiculo: string (nullable = true)
 |-- TIPO PERSONA: string (nullable = true)
 |-- S

In [4]:
import pyspark.sql.functions as func

# Reducimos las columnas que tenemos que utilizar mediante la Select
# y aplicamos las funciones necesarias a los datos
accRed=accidenteData\
  .filter ((func.substring(accidenteData.LESIVIDAD,1,1)=='H') | 
            (func.substring(accidenteData.LESIVIDAD,1,1)=='M') | 
            (accidenteData['TIPO PERSONA'] == 'CONDUCTOR'))\
  .select(
#        func.to_date(accidenteData.FECHA, 'dd/MM/yyyy HH:mm:ss').alias('fecha'), \
        func.to_date(accidenteData.FECHA, 'mm/dd/yyyy').alias('fecha'), \
#        accidenteData.FECHA.alias('fecha'),\
        accidenteData.DISTRITO.alias('distrito'), \
        accidenteData["Nº PARTE"].alias('idAccidente'), \
        accidenteData["TIPO PERSONA"].alias('tipoPersona'), \
        accidenteData["SEXO"].alias('sexo'), \
        func.substring(accidenteData.LESIVIDAD,1,2).alias('lesividad'), \
)

In [5]:
#Las columnas del Dataframe accRed quedan así
accRed.printSchema()
accRed.show(n=2)


root
 |-- fecha: date (nullable = true)
 |-- distrito: string (nullable = true)
 |-- idAccidente: string (nullable = true)
 |-- tipoPersona: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- lesividad: string (nullable = true)

+----------+--------------------+-----------+--------------------+------+---------+
|     fecha|            distrito|idAccidente|         tipoPersona|  sexo|lesividad|
+----------+--------------------+-----------+--------------------+------+---------+
|2018-01-01|USERA            ...|     2018/1|PEATON           ...|HOMBRE|       HG|
|2018-01-01|USERA            ...|     2018/1|           CONDUCTOR|HOMBRE|       IL|
+----------+--------------------+-----------+--------------------+------+---------+
only showing top 2 rows



In [6]:
# Agrupamos por fecha, distrito, accidente y generamos varias columnas contadoras 
accGrouped=accRed\
                .groupBy('fecha','distrito','idAccidente')\
                .agg(func.count(func.when(accRed.lesividad=='MT',1)).alias('muertos'),
                     func.count(func.when(accRed.lesividad=='HG',1)).alias('graves'),
                     func.count(func.when(accRed.lesividad=='HL',1)).alias('leves'),
                     func.count(func.when((accRed.sexo == 'MUJER') &
                                          (accRed.tipoPersona == 'CONDUCTOR'),1)).alias('condMujer'),
                     func.count(func.when((accRed.sexo == 'HOMBRE') &
                                          (accRed.tipoPersona == 'CONDUCTOR'),1)).alias('condHombre'),
                    )\
                .na.fill(0)\
                .sort ('fecha','distrito','idAccidente')

In [11]:
# Agrupamos por fecha, distrito, accidente y generamos varias columnas contadoras 
accGrCondFila=accGrouped.select ("fecha", "distrito", "idAccidente","muertos","graves","leves", "condMujer", "condHombre")\
                .withColumn("conductor",
                            func.when (accGrouped.condMujer == 0, "Hombre")\
                           .when (accGrouped.condHombre == 0, "Mujer")
                           .when (accGrouped.condHombre > 0,"Ambos")\
                )
accGrCondFila.toPandas()

,fecha,distrito,idAccidente,muertos,graves,leves,condMujer,condHombre,conductor
0,2018-01-01,ARGANZUELA,2018/1071,0,0,2,0,2,Hombre
1,2018-01-01,ARGANZUELA,2018/1086,0,0,1,0,2,Hombre
2,2018-01-01,ARGANZUELA,2018/1097,0,0,1,1,1,Ambos
3,2018-01-01,ARGANZUELA,2018/1163,0,0,1,0,2,Hombre
4,2018-01-01,ARGANZUELA,2018/1187,0,0,1,0,2,Hombre
5,2018-01-01,ARGANZUELA,2018/1188,0,1,0,0,2,Hombre
6,2018-01-01,ARGANZUELA,2018/1189,0,0,1,1,0,Mujer
7,2018-01-01,ARGANZUELA,2018/200,0,0,1,1,0,Mujer
8,2018-01-01,ARGANZUELA,2018/202,0,0,1,1,1,Ambos
9,2018-01-01,ARGANZUELA,2018/210,0,0,1,0,2,Hombre


In [12]:
#Las columnas del Dataframe accRed quedan así
accGrouped.toPandas()

,fecha,distrito,idAccidente,muertos,graves,leves,condMujer,condHombre
0,2018-01-01,ARGANZUELA,2018/1071,0,0,2,0,2
1,2018-01-01,ARGANZUELA,2018/1086,0,0,1,0,2
2,2018-01-01,ARGANZUELA,2018/1097,0,0,1,1,1
3,2018-01-01,ARGANZUELA,2018/1163,0,0,1,0,2
4,2018-01-01,ARGANZUELA,2018/1187,0,0,1,0,2
5,2018-01-01,ARGANZUELA,2018/1188,0,1,0,0,2
6,2018-01-01,ARGANZUELA,2018/1189,0,0,1,1,0
7,2018-01-01,ARGANZUELA,2018/200,0,0,1,1,0
8,2018-01-01,ARGANZUELA,2018/202,0,0,1,1,1
9,2018-01-01,ARGANZUELA,2018/210,0,0,1,0,2


In [12]:
accRes = accGrouped\
   .groupBy (accGrouped.fecha, accGrouped.distrito)\
   .agg (
            func.count(func.when(accGrouped.condMujer==0,1)).alias('condHombre'),
            func.count(func.when(accGrouped.condHombre==0,1)).alias('condMujer'),
            func.count(func.when((accGrouped.condMujer>0)&
                                 (accGrouped.condHombre>0),1)).alias('condMixto'),
            func.sum (func.when (accGrouped.condMujer==0,accGrouped.muertos)).alias('muertosCondHombre'),
            func.sum (func.when (accGrouped.condMujer==0,accGrouped.graves)).alias('gravesCondHombre'),
            func.sum (func.when (accGrouped.condMujer==0,accGrouped.leves)).alias('levesCondHombre'),
            func.sum (func.when (accGrouped.condHombre==0,accGrouped.muertos)).alias('muertosCondMujer'),
            func.sum (func.when (accGrouped.condHombre==0,accGrouped.graves)).alias('gravesCondMujer'),
            func.sum (func.when (accGrouped.condHombre==0,accGrouped.leves)).alias('levesCondMujer'),
            func.sum (accGrouped.muertos).alias('totalMuertos'),
            func.sum (accGrouped.graves).alias('totalGraves'),
            func.sum (accGrouped.leves).alias('totalLeves')
        )\
    .na.fill(0)\
    .orderBy (accGrouped.fecha, accGrouped.distrito)

In [13]:
df=accRes.toPandas()
df

,fecha,distrito,condHombre,condMujer,condMixto,muertosCondHombre,gravesCondHombre,levesCondHombre,muertosCondMujer,gravesCondMujer,levesCondMujer,totalMuertos,totalGraves,totalLeves
0,2018-01-01,ARGANZUELA,14,1,3,0,3,11,0,0,1,0,3,18
1,2018-01-01,BARAJAS,5,0,1,0,1,6,0,0,0,0,1,7
2,2018-01-01,CARABANCHEL,14,4,2,0,3,16,0,0,4,0,3,22
3,2018-01-01,CENTRO,20,3,4,0,0,26,0,0,3,0,0,34
4,2018-01-01,CHAMARTIN,13,2,7,0,2,14,0,0,2,0,2,28
5,2018-01-01,CHAMBERI,11,4,7,0,0,15,0,0,4,0,1,25
6,2018-01-01,CIUDAD LINEAL,11,5,2,0,1,12,0,0,5,0,1,18
7,2018-01-01,FUENCARRAL-EL PARDO,7,2,3,1,0,7,0,0,2,1,0,12
8,2018-01-01,HORTALEZA,10,5,4,0,3,8,0,0,7,0,4,19
9,2018-01-01,LATINA,8,3,6,0,0,13,0,0,4,0,0,25


#### Por comodidad para poder guardar todos los datos en un único fichero utilizamos la librería pandas.


In [14]:
df.to_csv('/home/ubuntu/Downloads/accidente_victimas_por_sexo_conductor.csv',sep=';')